In [1]:
import sys

import json
import itertools
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt

plt.style.use('../plt_params.mplstyle')

## Reading raw JSON data

Read into lists and reshape into numpy arrays

In [ ]:
input_dirs = list(Path('/Users/lucapaganin/Downloads/molet_output/test_2/').glob("mock*"))

ts_list  = []
lcs_list = []
dts_list = []

for d in input_dirs:
    dts_list.append(np.array([float(line) for line in (d / 'time_delays.txt').read_text().splitlines()]))

    with open(d / 'test_CAM_000_LC_continuous.json') as f:
        data_lc = json.load(f)
    
    time_domain = np.array(data_lc[0]['time'])    
    images_light_curves = np.array([x['signal'] for x in data_lc])
    
    lcs_list.append(images_light_curves)
    ts_list.append(time_domain)
    

time_domain = ts_list[0]

lcs = np.array(lcs_list)
dts = np.array(dts_list)

# lcs has shape n_imgs x n_ex_realiz x n_time_points
lcs = np.transpose(lcs, axes=(1,0,2))
# shape n_imgs x n_ex_realiz
dts = np.transpose(dts, axes=(1,0))

ATTENTION!! Here we delete the middle image light curves, since it is full of +inf values for some reason

In [ ]:
# ATTENTION!! Here we delete the middle row of the images light curves, since it is full of +inf values for some reason
lcs = np.delete(lcs, 2, axis=0)
# At the same time we have to remove the middle row for the delays, since it is related to the +inf signal values
dts = np.delete(dts, 2, axis=0)

Generate the 6 combinations

In [ ]:
mag_combos = []
dts_combos = []

n_images = lcs.shape[0]

for i, j in itertools.combinations(range(lcs.shape[0]), 2):
    mags_sum = -2.5*np.log10(10**(-lcs[i]/2.5) + 10**(-lcs[j]/2.5))
    dts_sum  = np.abs(dts[i] - dts[j])
    
    mag_combos.append(mags_sum)
    dts_combos.append(dts_sum)
    
    del mags_sum
    del dts_sum

mag_combos = np.array(mag_combos)
dts_combos = np.array(dts_combos)

Reshape data in the form X, y, where X is a matrix $N_{\rm entries} \times n_{\rm time}$ and y is a vector $N_{\rm entries} \times 1$

In [ ]:
n_combos, n_ex, n_time_samples = mag_combos.shape

mag_combos = mag_combos.reshape(n_combos*n_ex, n_time_samples)

dts_combos = dts_combos.reshape(n_combos*n_ex)

idxs_to_drop = np.where(np.any(np.isinf(mag_combos), axis=1))[0]

mag_combos = np.delete(mag_combos, idxs_to_drop, axis=0)
dts_combos = np.delete(dts_combos, idxs_to_drop, axis=0)

Save the data to a .npz compressed archive file

In [ ]:
np.savez_compressed('processed_data.npz', X=mag_combos, y=dts_combos)